# Demo 4: FP-Growth and DataStax Analytics
------
<img src="images/pixarMovies.jpg" width="500" height="500">


#### Dataset: https://grouplens.org/datasets/movielens/

## What are we trying to learn from this dataset? 

# QUESTION: Can FP-Growth be used to find which movies to recommend to our users?


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
import pandas
import cassandra
import pyspark
import re
import os
import random
import matplotlib.pyplot as plt
from random import randint, randrange
from IPython.display import display, Markdown
from pyspark.sql import SparkSession
from pyspark.ml.fpm import FPGrowth
from pyspark.sql import Row
from pyspark.sql.functions import collect_set
from pyspark.ml.feature import IndexToString, StringIndexer, VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
#Helper for pretty formatting for Spark DataFrames
def showDF(df, limitRows =  5, truncate = True):
    if(truncate):
        pandas.set_option('display.max_colwidth', 50)
    else:
        pandas.set_option('display.max_colwidth', -1)
    pandas.set_option('display.max_rows', limitRows)
    display(df.limit(limitRows).toPandas())
    pandas.reset_option('display.max_rows')

#### Helper function to have nicer formatting of Spark DataFrames

## Creating Tables and Loading Tables

<img src="images/dselogo.png" width="400" height="200">

In [ ]:
from cassandra.cluster import Cluster

cluster = Cluster(['dse'])
session = cluster.connect()

### Create Demo Keyspace 

In [ ]:
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS accelerate 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }""")

### Set keyspace 

In [ ]:
session.set_keyspace('accelerate')

### Create table called `movies`. Our PRIMARY will be a unique key (movieid) 

In [ ]:
query = "CREATE TABLE IF NOT EXISTS movies \
                                   (movieid int, title text, genres text, \
                                   PRIMARY KEY (movieid))"
session.execute(query)

### Create table called `movieRatings`. Our PRIMARY key will be a compositite key (userid, movieid).

In [ ]:
query = "CREATE TABLE IF NOT EXISTS movieratings \
                                   (userid int, movieid int, rating float, timestamp text, \
                                   PRIMARY KEY (userid, movieid))"
session.execute(query)

## Movies

* **Movieid**
* **Title**
* **Genres**

## Movie Ratings Table
### What do these of these 4 columns represent:

* **UserId**
* **MovieId**
* **Rating**
* **Timestamp**

### Load 2 Movie Dataset -- Movies and Movie Ratings Table
<img src="images/bttf3.jpg" width="500" height="300">

### Load Movie datasets from CSV file (rating_movies.csv, movies.csv)
* No clean up was requried! How nice :)

#### Insert all the Movie Data into the table `movies` and `movieratings`

In [ ]:
fileName = 'data/ratings.csv'
input_file = open(fileName, 'r')

for line in input_file:
    row = line.split(',')
    
    query = "INSERT INTO movieratings (userid, movieid, rating, timestamp)"
    query = query + " VALUES (%s, %s, %s, %s)"
    session.execute(query, (int(row[0]), int(row[1]), float(row[2]), row[3]))

In [ ]:
fileName = 'data/movies.csv'
input_file = open(fileName, 'r')

for line in input_file:
    row = line.split(',')
        
    query = "INSERT INTO movies (movieid, title, genres)"
    query = query + " VALUES (%s, %s, %s)"
    session.execute(query, (int(row[0]), row[1], row[2]))

## Machine Learning with Apache Spark
<img src="images/sparklogo.png" width="150" height="200">

#### Create a spark session that is connected to cassandra. From there load each table into a Spark Dataframe and take a count of the number of rows in each.

In [ ]:
spark = SparkSession.builder.appName('demo').master("local").getOrCreate()

movieDF = spark.read.format("org.apache.spark.sql.cassandra").options(table="movieratings", keyspace="accelerate").load()

print ("Table Row Count: ")
print (movieDF.count())

In [ ]:
showDF(movieDF)

## This dataset is not in the format we need it to be. We need it to be more in a transaction format. Each user and the list of movies they have reviewed. 

#### Remove the timestamp column since we will not be using that

In [ ]:
newMovieDF = movieDF.drop('timestamp')
showDF(newMovieDF)

#### Before we collect the set of movies for each user let's filter out any movies they rated below 3

In [ ]:
newestMovies = newMovieDF.filter("rating > 3")
showDF(newestMovies)

#### GroupBy the user id and create a collection set of all the movies they have rated and seen. 

In [ ]:
group_user = newestMovies.groupBy('userid').agg(collect_set('movieid').alias('moviesRated'))
group_user.show()


#### For FP-Growth the list needs to be a column named `items`

In [ ]:
df = group_user.withColumnRenamed("moviesRated", "items")
showDF(df)

In [ ]:
df.select('userid').distinct().count()

### FPGROWTH for  Recommendations
#### Use Apache Spark MLlib with FPGrowth to find Recommendation 
#### https://spark.apache.org/docs/latest/ml-frequent-pattern-mining.html
#### https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.fpm.FPGrowth

In [ ]:
fpGrowth = FPGrowth(itemsCol="items", minSupport=0.1, minConfidence=0.2)
model = fpGrowth.fit(df)
recommendDF=model.transform(df)
recommendDF.show()

#### If you have watched these movies `antecedent` then you will like this movie `consquent`

In [ ]:
# Display frequent itemsets.
#model.freqItemsets.show()

# Display generated association rules.
dfAssociation = model.associationRules

dfAssociation.show()

In [ ]:
print("If you like these movies: ")
print(list(dfAssociation.select('antecedent').first()))
print("Then you will like this movie:")
print(list(dfAssociation.select('consequent').first()))

movieYoulike = list(dfAssociation.select('antecedent').first())
movieToRecommend=list(dfAssociation.select('consequent').first())

#### Query database to get movie titles

In [ ]:
query = "select title from movies WHERE movieid="
query = query + str(movieYoulike[0][0])

rows = session.execute(query)
print(rows)

for user_row in rows:
    print (user_row.title)

query = "select title from movies WHERE movieid="
query = query + str(movieYoulike[0][1])

rows = session.execute(query)
print(rows)

for user_row in rows:
    print (user_row.title)

#### Then you will like this movie ... 

In [ ]:
query = "select title from movies WHERE movieid="
query = query + str(movieToRecommend[0][0])

rows = session.execute(query)
print(rows)

for user_row in rows:
    print (user_row.title)

In [ ]:
session.execute("""drop table movies""")
session.execute("""drop table movieratings""")